In [2]:
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import json
import pickle
from tensorflow.keras.optimizers import Adam

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from keras.optimizers import SGD
import random
from keras.models import load_model

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [7]:
print("~This is words list~")
print(words[3:5])
print("-"*50)
print("~This is documents list~")
print(documents[3:5])
print("-"*50)
print("~This is classes list~")
print(classes[3:10])

~This is words list~
['ada', 'orang']
--------------------------------------------------
~This is documents list~
[(['Hi', 'there'], 'greetings'), (['Hello'], 'greetings')]
--------------------------------------------------
~This is classes list~
['evening', 'night', 'goodbye', 'thanks', 'no-response', 'neutral-response', 'about']


In [8]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print("~Document Length~")
print(len(documents), "documents\n\n")
print("-"*100)

print("~Class Length~")
print(len(classes), "classes\n\n", classes)
print("-"*100)

print("~Word Length~")
print(len(words), "unique lemmatized words\n\n", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

~Document Length~
232 documents


----------------------------------------------------------------------------------------------------
~Class Length~
80 classes

 ['about', 'afternoon', 'anxious', 'ask', 'casual', 'creation', 'death', 'default', 'depressed', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greetings', 'happy', 'hate-me', 'hate-you', 'help', 'jokes', 'learn-mental-health', 'learn-more', 'location', 'meditation', 'mental-health-fact', 'morning', 'name', 'neutral-response', 'night', 'no-approach', 'no-response', 'not-talking', 'pandora-useful', 'problem', 'repeat', 'sad', 'scared', 'skill', 'sleep', 'something-else', 'stressed', 'stupid', 'suicide', 't

In [9]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


In [10]:
model = Sequential()
model.add(Dense(258, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(len(train_y[0]), activation='softmax'))

adam_optimizer = Adam()
# sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot.h5', hist)
print("\n")
print("*" * 50)
print("\nModel Created Successfully")


Epoch 1/200
47/47 [==============================] - 1s 3ms/step - loss: 4.3916 - accuracy: 0.0302
Epoch 2/200
47/47 [==============================] - 0s 3ms/step - loss: 4.2487 - accuracy: 0.0948
Epoch 3/200
47/47 [==============================] - 0s 3ms/step - loss: 4.1397 - accuracy: 0.0819
Epoch 4/200
47/47 [==============================] - 0s 3ms/step - loss: 3.9775 - accuracy: 0.0948
Epoch 5/200
47/47 [==============================] - 0s 5ms/step - loss: 3.7818 - accuracy: 0.1466
Epoch 6/200
47/47 [==============================] - 0s 7ms/step - loss: 3.5775 - accuracy: 0.1767
Epoch 7/200
47/47 [==============================] - 0s 4ms/step - loss: 3.4422 - accuracy: 0.2155
Epoch 8/200
47/47 [==============================] - 0s 6ms/step - loss: 3.2179 - accuracy: 0.2845
Epoch 9/200
47/47 [==============================] - 0s 7ms/step - loss: 2.9543 - accuracy: 0.3190
Epoch 10/200
47/47 [==============================] - 0s 6ms/step - loss: 2.7880 - accuracy: 0.3190
Epoch 11/